# Root Cause Analysis 

This code corresponds to the publication "Counterfactual Root Cause Analysis via Anomaly Detection and Causal Graphs" at INDIN 2023.
We rate pottential prior-known root causes by how they match anomaly patterns generated by a causal graph using the Jaccard Root Cause Score (JRCS). We demonstrate this using an industrial setup.
In this setup, an object is moved by a conveyor to a gripping position, it is gripped by a magnetic arm attached to cooperating robotic grippers and is hauled into a storage shelf position.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx
from collections.abc import Callable

## 1. Create Causal Directed Graph

In [2]:
Causal_DiGraph = nx.DiGraph()
edge_tuples = [
    ('PlateNotInPlace (Boolean)', 'plateInGrippingPosition (Boolean)'),
    ('beltSpeed (Centimeters/Second)', 'plateInGrippingPosition (Boolean)'),
    ('plateInGrippingPosition (Boolean)', 'SensorBelt_detectionDistance (Meters)'),
    ('beltIsBroken (Boolean)', 'beltSpeed (Centimeters/Second)'),
    ('plateInGrippingPosition (Boolean)', 'plateIsGripped (Boolean)'),
    ('robotsAreBroken (Boolean)', 'robotsAreMoving (Boolean)'),
    ('robotsAreMoving (Boolean)', 'robotsInDepositPosition (Boolean)'),
    ('robotsAreMoving (Boolean)', 'robotsInGrippingPosition (Boolean)'),
    ('magneticPowerGripper (Gauss)', 'plateIsGripped (Boolean)'),
    ('robotsInGrippingPosition (Boolean)', 'plateIsGripped (Boolean)'),
    ('plateIsGripped (Boolean)', 'plateDetectedOnShelf (Boolean)'),
    ('plateInGrippingPosition (Boolean)', 'plateIsGripped (Boolean)'),
    ('plateIsGripped (Boolean)', 'plateDetectedOnShelf (Boolean)'),
    ('robotsAreMoving (Boolean)', 'plateDetectedOnShelf (Boolean)'),
    ('powerConsumptionGripper (Watt)', 'magneticPowerGripper (Gauss)'),
    ('plateIsGripped (Boolean)', 'plateDetectedOnShelf (Boolean)'),
    ('magneticPowerGripper (Gauss)', 'plateIsGripped (Boolean)'),
    ('plateIsGripped (Boolean)', 'plateDetectedOnShelf (Boolean)'),
    ('magneticPowerTooLowForPlate (Boolean)', 'magneticPowerGripper (Gauss)'),
    ('sensorShelf_detectionDistance (Meters)', 'plateDetectedOnShelf (Boolean)'),
    ('shelfSensorIsBroken (Boolean)', 'sensorShelf_detectionDistance (Meters)'),
    ('robotsInDepositPosition (Boolean)', 'plateDetectedOnShelf (Boolean)'),
    ('robotsInGrippingPosition (Boolean)', 'RobotsWereInGrippingPosition (Boolean)'),
    ('robotsInDepositPosition (Boolean)', 'RobotsWereInDepositPosition (Boolean)'),
    ('robotsInGrippingPosition (Boolean)', 'robot1_jointAngle1 (Degrees)'),
    ('robotsInGrippingPosition (Boolean)', 'robot1_jointAngle2 (Degrees)'),
    ('robotsInGrippingPosition (Boolean)', 'robot1_jointAngle3 (Degrees)'),
    ('robotsInGrippingPosition (Boolean)', 'robot1_jointAngle4 (Degrees)'),
    ('robotsInGrippingPosition (Boolean)', 'robot1_jointAngle5 (Degrees)'),
    ('robotsInGrippingPosition (Boolean)', 'robot1_jointAngle6 (Degrees)'),
    ('robotsInGrippingPosition (Boolean)', 'robot1_jointAngle7 (Degrees)'),
    ('robotsInGrippingPosition (Boolean)', 'robot2_jointAngle1 (Degrees)'),
    ('robotsInGrippingPosition (Boolean)', 'robot2_jointAngle2 (Degrees)'),
    ('robotsInGrippingPosition (Boolean)', 'robot2_jointAngle3 (Degrees)'),
    ('robotsInGrippingPosition (Boolean)', 'robot2_jointAngle4 (Degrees)'),
    ('robotsInGrippingPosition (Boolean)', 'robot2_jointAngle5 (Degrees)'),
    ('robotsInGrippingPosition (Boolean)', 'robot2_jointAngle6 (Degrees)'),
    ('robotsInGrippingPosition (Boolean)', 'robot2_jointAngle7 (Degrees)'),
    ('robotsInDepositPosition (Boolean)', 'robot1_jointAngle1 (Degrees)'),
    ('robotsInDepositPosition (Boolean)', 'robot1_jointAngle2 (Degrees)'),
    ('robotsInDepositPosition (Boolean)', 'robot1_jointAngle3 (Degrees)'),
    ('robotsInDepositPosition (Boolean)', 'robot1_jointAngle4 (Degrees)'),
    ('robotsInDepositPosition (Boolean)', 'robot1_jointAngle5 (Degrees)'),
    ('robotsInDepositPosition (Boolean)', 'robot1_jointAngle6 (Degrees)'),
    ('robotsInDepositPosition (Boolean)', 'robot1_jointAngle7 (Degrees)'),
    ('robotsInDepositPosition (Boolean)', 'robot2_jointAngle1 (Degrees)'),
    ('robotsInDepositPosition (Boolean)', 'robot2_jointAngle2 (Degrees)'),
    ('robotsInDepositPosition (Boolean)', 'robot2_jointAngle3 (Degrees)'),
    ('robotsInDepositPosition (Boolean)', 'robot2_jointAngle4 (Degrees)'),
    ('robotsInDepositPosition (Boolean)', 'robot2_jointAngle5 (Degrees)'),
    ('robotsInDepositPosition (Boolean)', 'robot2_jointAngle6 (Degrees)'),
    ('robotsInDepositPosition (Boolean)', 'robot2_jointAngle7 (Degrees)'),
    ('robotsAreMoving (Boolean)', 'robot1_joint1_velocity (Degrees/Seconds)'),
    ('robotsAreMoving (Boolean)', 'robot1_joint2_velocity (Degrees/Seconds)'),
    ('robotsAreMoving (Boolean)', 'robot1_joint3_velocity (Degrees/Seconds)'),
    ('robotsAreMoving (Boolean)', 'robot1_joint4_velocity (Degrees/Seconds)'),
    ('robotsAreMoving (Boolean)', 'robot1_joint5_velocity (Degrees/Seconds)'),
    ('robotsAreMoving (Boolean)', 'robot1_joint6_velocity (Degrees/Seconds)'),
    ('robotsAreMoving (Boolean)', 'robot1_joint7_velocity (Degrees/Seconds)'),
    ('robotsAreMoving (Boolean)', 'robot2_joint1_velocity (Degrees/Seconds)'),
    ('robotsAreMoving (Boolean)', 'robot2_joint2_velocity (Degrees/Seconds)'),
    ('robotsAreMoving (Boolean)', 'robot2_joint3_velocity (Degrees/Seconds)'),
    ('robotsAreMoving (Boolean)', 'robot2_joint4_velocity (Degrees/Seconds)'),
    ('robotsAreMoving (Boolean)', 'robot2_joint5_velocity (Degrees/Seconds)'),
    ('robotsAreMoving (Boolean)', 'robot2_joint6_velocity (Degrees/Seconds)'),
    ('robotsAreMoving (Boolean)', 'robot2_joint7_velocity (Degrees/Seconds)'),
    ('plateIsGripped (Boolean)', 'torque (Newtonmeter)'),
    ('plateDetectedOnShelf (Boolean)', 'sensorShelf_detectionDistance (Meters)')
]
Causal_DiGraph.add_edges_from(edge_tuples)

## 2. Provide Known Potential Root Causes and Sensor Measurement Information

In [3]:
rootcauses = ['magneticPowerTooLowForPlate (Boolean)', 'beltIsBroken (Boolean)', 'PlateNotInPlace (Boolean)',
              'robotsAreBroken (Boolean)', 'shelfSensorIsBroken (Boolean)']

measurables = ['SensorBelt_detectionDistance (Meters)',
               'robot1_jointAngle1 (Degrees)',
               'robot1_jointAngle2 (Degrees)',
               'robot1_jointAngle3 (Degrees)',
               'robot1_jointAngle4 (Degrees)',
               'robot1_jointAngle5 (Degrees)',
               'robot1_jointAngle6 (Degrees)',
               'robot1_jointAngle7 (Degrees)',
               'robot2_jointAngle1 (Degrees)',
               'robot2_jointAngle2 (Degrees)',
               'robot2_jointAngle3 (Degrees)',
               'robot2_jointAngle4 (Degrees)',
               'robot2_jointAngle5 (Degrees)',
               'robot2_jointAngle6 (Degrees)',
               'robot2_jointAngle7 (Degrees)',
               'robot1_joint1_velocity (Degrees/Seconds)',
               'robot1_joint2_velocity (Degrees/Seconds)',
               'robot1_joint3_velocity (Degrees/Seconds)',
               'robot1_joint4_velocity (Degrees/Seconds)',
               'robot1_joint5_velocity (Degrees/Seconds)',
               'robot1_joint6_velocity (Degrees/Seconds)',
               'robot1_joint7_velocity (Degrees/Seconds)',
               'robot2_joint1_velocity (Degrees/Seconds)',
               'robot2_joint2_velocity (Degrees/Seconds)',
               'robot2_joint3_velocity (Degrees/Seconds)',
               'robot2_joint4_velocity (Degrees/Seconds)',
               'robot2_joint5_velocity (Degrees/Seconds)',
               'robot2_joint6_velocity (Degrees/Seconds)',
               'robot2_joint7_velocity (Degrees/Seconds)',
               'beltSpeed (Centimeters/Second)',
               'torque (Newtonmeter)',
               'sensorShelf_detectionDistance (Meters)',
               'powerConsumptionGripper (Watt)'
               ]

## 3. Define Graph Operations

In [4]:
def get_all_related_variables(variable_name: str, get_related_vars: Callable) -> list[str]:
    related_vars = get_related_vars(variable_name)
    vars_with_potential_relations = get_related_vars(variable_name)
    inspected_vars = []
    while vars_with_potential_relations:
        inspection_var = vars_with_potential_relations[0]
        inspected_vars += [inspection_var]

        direct_related_vars = get_related_vars(inspection_var)
        if direct_related_vars:
            for var in direct_related_vars:
                if var not in inspected_vars:
                    vars_with_potential_relations += [var]
                    related_vars += [var]
        vars_with_potential_relations.remove(inspection_var)
    return related_vars

In [5]:
def get_direct_child_vars(var: str) -> list[list[str]]:
    edge_list = Causal_DiGraph.out_edges(var)
    return [edge_tuple[1] for edge_tuple in edge_list]

In [6]:
def get_direct_parent_vars(var: str) -> list[list[str]]:
    edge_list = Causal_DiGraph.in_edges(var)
    return [edge_tuple[0] for edge_tuple in edge_list]

In [7]:
def find_all_effected_variables_for(variable_name):
    causes = []
    variables_to_inspect= [variable_name]
    inspected_variables = []
    while variables_to_inspect != []:
        inspected_variable = variables_to_inspect[0]
        inspected_variables = inspected_variables + [inspected_variable]
        
        
        parent_variables = find_effected_variables_for(inspected_variable)
        if variables_to_inspect == []: break
        
        if parent_variables == []:
            causes = causes + [variables_to_inspect[0]]
            
        if parent_variables != []:
            for cause in parent_variables:
                if cause not in inspected_variables:
                    variables_to_inspect = variables_to_inspect + [cause]
                    causes = causes + [cause]
        variables_to_inspect.remove(inspected_variable)
                    
        
    causes = list(set(causes))
    return causes

In [8]:
def Jaccard_similarity(list1: list[str], list2: list[str]):
    set1 = set(list1)
    set2 = set(list2)
    nominator = set1.intersection(set2)
    denominator = set1.union(set2)
    return len(nominator) / len(denominator)

## 4. Score all Root Causes for Detected Anomalies

In [9]:
print(f"\033[1m{'Measured variable':{60}} {'Potential affecting root cause':{20}} \033[0m  \n")
for measured_variable in measurables:
    children_vars = get_all_related_variables(measured_variable, get_direct_parent_vars)
    pot_rootcauses = [child for child in children_vars if child in rootcauses]
    print(f"{measured_variable:{60}}{' '.join(pot_rootcauses):{20}} \n")

Measured variable                                            Potential affecting root cause   

SensorBelt_detectionDistance (Meters)                       PlateNotInPlace (Boolean) beltIsBroken (Boolean) 

robot1_jointAngle1 (Degrees)                                robotsAreBroken (Boolean) robotsAreBroken (Boolean) 

robot1_jointAngle2 (Degrees)                                robotsAreBroken (Boolean) robotsAreBroken (Boolean) 

robot1_jointAngle3 (Degrees)                                robotsAreBroken (Boolean) robotsAreBroken (Boolean) 

robot1_jointAngle4 (Degrees)                                robotsAreBroken (Boolean) robotsAreBroken (Boolean) 

robot1_jointAngle5 (Degrees)                                robotsAreBroken (Boolean) robotsAreBroken (Boolean) 

robot1_jointAngle6 (Degrees)                                robotsAreBroken (Boolean) robotsAreBroken (Boolean) 

robot1_jointAngle7 (Degrees)                                robotsAreBroken (Boolean) robotsAreBroken (Boolea

In [10]:
print(f"\033[1m{'Potential root cause':{40}} {'Potential Affected Measured Values':{20}} \033[0m  \n")
for potential_root_cause in rootcauses:
    children_vars = get_all_related_variables(potential_root_cause, get_direct_child_vars)
    affected_measurables = [child for child in children_vars if child in measurables]
    print(f"{potential_root_cause:{40}}{', '.join(affected_measurables):{20}} \n")

Potential root cause                     Potential Affected Measured Values   

magneticPowerTooLowForPlate (Boolean)   torque (Newtonmeter), sensorShelf_detectionDistance (Meters) 

beltIsBroken (Boolean)                  beltSpeed (Centimeters/Second), SensorBelt_detectionDistance (Meters), torque (Newtonmeter), sensorShelf_detectionDistance (Meters) 

PlateNotInPlace (Boolean)               SensorBelt_detectionDistance (Meters), torque (Newtonmeter), sensorShelf_detectionDistance (Meters) 

robotsAreBroken (Boolean)               robot1_joint1_velocity (Degrees/Seconds), robot1_joint2_velocity (Degrees/Seconds), robot1_joint3_velocity (Degrees/Seconds), robot1_joint4_velocity (Degrees/Seconds), robot1_joint5_velocity (Degrees/Seconds), robot1_joint6_velocity (Degrees/Seconds), robot1_joint7_velocity (Degrees/Seconds), robot2_joint1_velocity (Degrees/Seconds), robot2_joint2_velocity (Degrees/Seconds), robot2_joint3_velocity (Degrees/Seconds), robot2_joint4_velocity (Degrees/Seconds),

In [11]:
# Anomalies should be detected before root cause analysis
detected_faults = ['SensorBelt_detectionDistance (Meters)', 'torque (Newtonmeter)']

print(f"\033[1m{'Potential root cause':{50}} {'JRCS value':{20}} \033[0m  \n")
for potential_root_cause in rootcauses:
    children_vars = get_all_related_variables(potential_root_cause, get_direct_child_vars)
    affected_measurables = [child for child in children_vars if child in measurables]
    rootcause_JRCS = Jaccard_similarity(detected_faults, affected_measurables)
    print(f"{potential_root_cause:{40}}{rootcause_JRCS:{20}.3f}  \n")

Potential root cause                               JRCS value             

magneticPowerTooLowForPlate (Boolean)                  0.333  

beltIsBroken (Boolean)                                 0.500  

PlateNotInPlace (Boolean)                              0.667  

robotsAreBroken (Boolean)                              0.032  

shelfSensorIsBroken (Boolean)                          0.000  

